In [1]:
intcode = [3,225,1,225,6,6,1100,1,238,225,104,0,1002,36,25,224,1001,224,-2100,224,4,224,1002,223,8,223,101,1,224,224,1,223,224,223,1102,31,84,225,1102,29,77,225,1,176,188,224,101,-42,224,224,4,224,102,8,223,223,101,3,224,224,1,223,224,223,2,196,183,224,1001,224,-990,224,4,224,1002,223,8,223,101,7,224,224,1,224,223,223,102,14,40,224,101,-1078,224,224,4,224,1002,223,8,223,1001,224,2,224,1,224,223,223,1001,180,64,224,101,-128,224,224,4,224,102,8,223,223,101,3,224,224,1,223,224,223,1102,24,17,224,1001,224,-408,224,4,224,1002,223,8,223,101,2,224,224,1,223,224,223,1101,9,66,224,1001,224,-75,224,4,224,1002,223,8,223,1001,224,6,224,1,223,224,223,1102,18,33,225,1101,57,64,225,1102,45,11,225,1101,45,9,225,1101,11,34,225,1102,59,22,225,101,89,191,224,1001,224,-100,224,4,224,1002,223,8,223,1001,224,1,224,1,223,224,223,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,8,226,677,224,1002,223,2,223,1006,224,329,1001,223,1,223,108,226,226,224,1002,223,2,223,1006,224,344,1001,223,1,223,7,677,226,224,102,2,223,223,1005,224,359,101,1,223,223,7,226,677,224,102,2,223,223,1006,224,374,101,1,223,223,1008,677,226,224,1002,223,2,223,1006,224,389,101,1,223,223,8,677,677,224,1002,223,2,223,1005,224,404,101,1,223,223,8,677,226,224,102,2,223,223,1005,224,419,1001,223,1,223,1107,677,226,224,102,2,223,223,1005,224,434,1001,223,1,223,1107,226,677,224,1002,223,2,223,1006,224,449,1001,223,1,223,107,677,226,224,1002,223,2,223,1005,224,464,1001,223,1,223,1008,677,677,224,1002,223,2,223,1006,224,479,1001,223,1,223,1108,677,226,224,1002,223,2,223,1006,224,494,1001,223,1,223,1108,677,677,224,1002,223,2,223,1006,224,509,1001,223,1,223,107,677,677,224,1002,223,2,223,1005,224,524,101,1,223,223,1007,677,226,224,102,2,223,223,1005,224,539,1001,223,1,223,1107,226,226,224,1002,223,2,223,1006,224,554,1001,223,1,223,1008,226,226,224,1002,223,2,223,1006,224,569,101,1,223,223,1108,226,677,224,1002,223,2,223,1006,224,584,101,1,223,223,108,677,677,224,1002,223,2,223,1006,224,599,1001,223,1,223,1007,677,677,224,102,2,223,223,1006,224,614,101,1,223,223,107,226,226,224,102,2,223,223,1006,224,629,101,1,223,223,1007,226,226,224,102,2,223,223,1005,224,644,1001,223,1,223,108,226,677,224,102,2,223,223,1005,224,659,1001,223,1,223,7,677,677,224,102,2,223,223,1006,224,674,1001,223,1,223,4,223,99,226]

In [2]:
from operator import add, mul

def compute_intcode(intcode, input_values=None):
    def read():
        mode3, mode2, mode1, code1, code2 = map(int, f'{intcode[pointer]:05}')
        return code1 * 10 + code2, mode1, mode2, mode3
    
    def getv(mode, shift):
        return intcode[pointer + shift] if mode else intcode[intcode[pointer + shift]]
    
    def setv(mode, value, shift=3):
        intcode[pointer + shift if mode else intcode[pointer + shift]] = value 
    
    results = []
    pointer = 0
    opcode, mode1, mode2, mode3 = read()
    while opcode != 99:
        if opcode == 1:
            setv(mode3, add(getv(mode1, 1), getv(mode2, 2)))
            pointer += 4
        elif opcode == 2:
            setv(mode3, mul(getv(mode1, 1), getv(mode2, 2)))
            pointer += 4
        elif opcode == 3:
            value = input_values.pop(0)
            setv(mode1, value, shift=1)
            pointer += 2
        elif opcode == 4:
            value = getv(mode1, 1)
            results.append(value)
            if value:
                input_value = (yield value)
                if input_value is not None:
                    input_values.append(input_value)
            pointer += 2
        elif opcode == 5:
            pointer = getv(mode2, 2) if getv(mode1, 1) else pointer + 3
        elif opcode == 6:
            pointer = getv(mode2, 2) if not getv(mode1, 1) else pointer + 3
        elif opcode == 7:
            setv(mode3, 1) if getv(mode1, 1) < getv(mode2, 2) else setv(mode3, 0)
            pointer += 4
        elif opcode == 8:
            setv(mode3, 1) if getv(mode1, 1) == getv(mode2, 2) else setv(mode3, 0)
            pointer += 4
        opcode, mode1, mode2, mode3 = read()
    return results[-1]

In [3]:
part1 = compute_intcode(intcode[:], input_values=[1])
next(part1)

3122865

In [4]:
part2 = compute_intcode(intcode[:], input_values=[5])
next(part2)

773660